<a href="https://colab.research.google.com/github/bryanbayup/petpoint/blob/main/PetpointWithGeneralConver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Sastrawi

In [2]:
import re
import string
import pandas as pd
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

Membaca Dataset dan Preprocessing Teks

In [3]:
# Membaca dataset penyakit
df_disease = pd.read_csv('disease_classification.csv')

In [4]:
# Preprocessing dataset
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.strip()
    return text

In [5]:
def remove_stopwords(text):
    factory = StopWordRemoverFactory()
    stopword_remover = factory.create_stop_word_remover()
    cleaned_text = stopword_remover.remove(text)
    return cleaned_text

In [6]:
df_disease['Cleaned_Gejala'] = df_disease['Gejala'].apply(clean_text).apply(remove_stopwords)
df_disease['Cleaned_Penanganan'] = df_disease['Penanganan Pertama'].apply(clean_text).apply(remove_stopwords)

In [7]:
# Mengubah nama penyakit menjadi label numerik
le_disease = LabelEncoder()
df_disease['Label_Penyakit'] = le_disease.fit_transform(df_disease['Nama Penyakit'])

In [8]:
# Menggabungkan teks gejala
all_text_disease = df_disease['Cleaned_Gejala'].tolist()
vectorizer_disease = TfidfVectorizer()
X_disease = vectorizer_disease.fit_transform(all_text_disease)
y_disease = df_disease['Label_Penyakit']

In [9]:
# Membagi data untuk penyakit
X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(
    X_disease, y_disease, test_size=0.2, random_state=42)

In [10]:
# Membangun model untuk penyakit
model_disease = Sequential()
model_disease.add(Dense(128, input_shape=(X_train_d.shape[1],), activation='relu'))
model_disease.add(Dropout(0.5))
model_disease.add(Dense(64, activation='relu'))
model_disease.add(Dense(len(le_disease.classes_), activation='softmax'))
model_disease.compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])
model_disease.fit(X_train_d, y_train_d,
                  epochs=50,
                  batch_size=64,
                  validation_data=(X_test_d, y_test_d))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 365ms/step - accuracy: 0.0336 - loss: 2.3904 - val_accuracy: 0.2250 - val_loss: 2.2936
Epoch 2/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 78ms/step - accuracy: 0.1980 - loss: 2.3017 - val_accuracy: 0.4500 - val_loss: 2.2093
Epoch 3/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step - accuracy: 0.4125 - loss: 2.2104 - val_accuracy: 0.7500 - val_loss: 2.1306
Epoch 4/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 103ms/step - accuracy: 0.6199 - loss: 2.1269 - val_accuracy: 0.7500 - val_loss: 2.0472
Epoch 5/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.6961 - loss: 2.0496 - val_accuracy: 1.0000 - val_loss: 1.9562
Epoch 6/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step - accuracy: 0.7664 - loss: 1.9785 - val_accuracy: 1.0000 - val_loss: 1.8605
Epoch 7/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - accuracy: 0.8230 - loss: 1.8721 - val_accuracy: 1.0000 - val_loss: 1.7544
Epoch 8/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.8852 - loss: 1.7416 - val_accuracy: 1.0000 - val_loss: 1.6

In [11]:
# Dataset percakapan umum
df_conversation = pd.read_csv('general_conversation.csv')

In [24]:
# Preprocessing dataset percakapan umum
df_conversation['Cleaned_Input'] = df_conversation['Input'].apply(clean_text).apply(remove_stopwords)

In [34]:
# Membuat mapping dictionary untuk percakapan umum
conversation_dict = dict(zip(df_conversation['Cleaned_Input'], df_conversation['Output']))

In [35]:
def get_general_response(user_input):
    processed_input = clean_text(user_input)
    processed_input = remove_stopwords(processed_input)
    response = conversation_dict.get(processed_input, None)
    return response

In [36]:
# Fungsi deteksi hewan
def detect_animal_type(user_input):
    animal_types = ['kucing', 'anjing']
    for animal in animal_types:
        if animal in user_input.lower():
            return animal
    return None

In [37]:
# Daftar gejala
symptoms_keywords = ['muntah', 'demam', 'batuk', 'diare', 'kejang', 'lemas',
                     'pilek', 'bersin', 'luka', 'lesu', 'gatal', 'tidak nafsu makan', 'dehidrasi']

In [38]:
def is_symptom_input(user_input):
    return any(keyword in user_input.lower() for keyword in symptoms_keywords)

In [39]:
def is_no_more_symptoms(user_input):
    negations = ['tidak', 'nggak', 'enggak', 'gak', 'ga', 'udah', 'cukup', 'sudah', 'tidak ada']
    return any(word in user_input.lower() for word in negations)

In [40]:
# set Variabel global
current_animal = None
symptoms_list = []
conversation_state = 'initial'

In [41]:
def chatbot_response(user_input):
    global current_animal, symptoms_list, conversation_state

    # Preprocessing input pengguna
    processed_input = clean_text(user_input)
    processed_input = remove_stopwords(processed_input)

    if conversation_state == 'asking_symptoms':
        if is_no_more_symptoms(user_input):
            if symptoms_list:
                combined_symptoms = ' '.join(symptoms_list)
                vectorized_symptoms = vectorizer_disease.transform([combined_symptoms])
                prediction = model_disease.predict(vectorized_symptoms)
                predicted_label = prediction.argmax(axis=-1)[0]
                disease_name = le_disease.inverse_transform([predicted_label])[0]
                df_animal = df_disease[df_disease['Nama Hewan'].str.lower() == current_animal]
                first_aid = df_animal[df_animal['Nama Penyakit'] == disease_name]['Penanganan Pertama'].iloc[0]
                response = f"Berdasarkan gejala yang Anda sebutkan, sepertinya {current_animal} Anda mengalami {disease_name}. {first_aid}"
                # Reset percakapan
                conversation_state = 'initial'
                current_animal = None
                symptoms_list = []
                return response
            else:
                conversation_state = 'initial'
                current_animal = None
                symptoms_list = []
                return "Anda tidak menyebutkan gejala apapun. Ada yang bisa saya bantu?"
        else:
            symptoms_list.append(user_input)
            return "Apakah ada gejala lain yang dialami hewan Anda? Jika tidak ada, silakan katakan 'tidak ada'."
    elif conversation_state == 'asking_animal':
        animal = detect_animal_type(user_input)
        if animal:
            current_animal = animal
            conversation_state = 'asking_symptoms'
            return f"Apa gejala yang dialami {current_animal} Anda?"
        else:
            return "Maaf, saya hanya bisa memberikan saran untuk kucing atau anjing. Hewan apa yang Anda miliki?"
    else:
        # Cek apakah input adalah percakapan umum
        response = get_general_response(user_input)
        if response:
            return response
        # Cek apakah input terkait dengan hewan sakit
        elif 'sakit' in user_input.lower() or is_symptom_input(user_input):
            animal = detect_animal_type(user_input)
            if animal:
                current_animal = animal
                conversation_state = 'asking_symptoms'
                return f"Apa gejala yang dialami {current_animal} Anda?"
            else:
                conversation_state = 'asking_animal'
                return "Hewan apa yang Anda miliki? (kucing/anjing)"
        else:
            return "Maaf, saya tidak mengerti. Bisa dijelaskan lebih lanjut?"

In [ ]:
# Loop interaksi dengan pengguna
print("Chatbot: Halo, ada yang bisa saya bantu?")
while True:
    user_input = input("Anda: ")
    if user_input.lower() in ['exit', 'quit', 'keluar', 'bye', 'sampai jumpa']:
        print("Chatbot: Terima kasih, semoga membantu!")
        break
    response = chatbot_response(user_input)
    print("Chatbot:", response)

Chatbot: Halo, ada yang bisa saya bantu?
Anda: halo
Chatbot: Halo! Ada yang bisa saya bantu?
Anda: siapa kamu
Chatbot: Saya adalah Alexa, chatbot yang siap membantu Anda.
Anda: kucing saya sakit
Chatbot: Apa gejala yang dialami kucing Anda?
Anda: diare
Chatbot: Apakah ada gejala lain yang dialami hewan Anda? Jika tidak ada, silakan katakan 'tidak ada'.
Anda: demam
Chatbot: Apakah ada gejala lain yang dialami hewan Anda? Jika tidak ada, silakan katakan 'tidak ada'.
Anda: cukup
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Chatbot: Berdasarkan gejala yang Anda sebutkan, sepertinya kucing Anda mengalami Feline Leukemia Virus. Isolasi kucing yang terinfeksi, berikan perawatan suportif, konsultasi dokter hewan. Pastikan untuk konsultasi dengan dokter hewan.
Anda: anjing saya sakit
Chatbot: Apa gejala yang dialami anjing Anda?
Anda: muntah
Chatbot: Apakah ada gejala lain yang dialami hewan Anda? Jika tidak ada, silakan katakan 'tidak ada'.
Anda: dehidrasi dan tidak nafsu makan
1/1 ━━━━━━━━━━━━━━━━━━